In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import math
import cv2
from sklearn import model_selection


In [2]:
model_name = 'model'
data_path = './data/'
batch_size = 8
samples_epoch = batch_size * 6
n_epochs = 1
df = pd.read_csv(data_path + 'driving_log.csv')
train, valid = model_selection.train_test_split(df, test_size=0.1)

In [ ]:
model.pred_steering().summary()